Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Abinesh Senthil Kumar"
COLLABORATORS = ""

---

# Lab 2 Introduction


In this lab you will use parse a vcf file to extracts parts of it and load it into a database. 
The fields you will be parsing from the vcf file are: 
```
CHROM
POS	
ID	
REF	
ALT	
QUAL	
FILTER
```
and from the INFO column, the following fields:
```
1000g2015aug_all
ExAC_ALL
FATHMM_pred
LRT_pred
MetaLR_pred
MetaSVM_pred
MutationAssessor_pred
MutationTaster_pred
PROVEAN_pred
Polyphen2_HDIV_pred
Polyphen2_HVAR_pred
SIFT_pred
fathmm-MKL_coding_pred.
```
The fields:
```
FATHMM_pred
LRT_pred
MetaLR_pred
MetaSVM_pred
MutationAssessor_pred
MutationTaster_pred
PROVEAN_pred
Polyphen2_HDIV_pred
Polyphen2_HVAR_pred
SIFT_pred
fathmm-MKL_coding_pred
```
are predictor fields. They use a letter to indicate whether a given variation is harmful or not.

NOTE: use the helper functions in cell2. To use them, you have to run CELL 2! 


In [2]:
## Helper functions

import os
import sqlite3
from sqlite3 import Error
import gzip

def create_connection(db_file, delete_db=False):
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)



# Part 1 (10 pts)

In part1 you will open the gzipped vcf file using the python gzip module and figure out all the possible values for the predictor fields. 

You do not have to unzip the file. Use `with gzip.open(filename,'rt') as fp:` to read one line at a time. 



In [3]:
def get_predictor_values(filename):
    
    import gzip
    with gzip.open(filename, 'rt') as fp:
        file = list(fp)
    ################
        def format_sample_fields(format_field, sample_field):
   
            samplevalues = []
            samplekey = []
            for i,v in sample_field.items():
                samplevalues.append(v)
                samplekey.append(i)

            format_field = format_field.split(":")


            samplevalues = [i.split(":") for i in samplevalues]

            c = []

            for v in samplevalues:
                c.append(v)

            dict2 = []

            for z in c:
        
                dict2.append(dict(zip(format_field , z)))
    
            finaldict = dict(zip(samplekey,dict2))
    
            return finaldict

        #############
        def create_dict_from_line(header, line):
            keyas = []

            for i in header:
                if i == "FORMAT":
                    break
                keyas.append(i.strip())

            keyas1 = []
            for i in header:
                keyas1.append(i.strip())
    
            keyas2 = keyas1[(len(keyas)+1):]

            dictivaly = line.split("\t")

            dictivaly1 = dictivaly[:len(keyas)]

            dictionary = dict(zip(keyas,dictivaly1))

            dictionary8th = str(dictivaly[len(keyas)])

            dictionary2 = dict(zip(keyas2,dictivaly[(len(keyas)+1):]))

            formatoutput1 = format_sample_fields(dictionary8th,dictionary2)

            dict2nd = {'SAMPLE':formatoutput1}
    
            dictionary.update(dict2nd)
    
   
            return dictionary
        ############
        def read_vcf_file(filename):
    
        #file = open(filename, 'r')
    
            listy = [i.strip() for i in file if i.strip()]
            headery = [i.strip() for i in listy if i.startswith('#CHROM')]
  
            line1 = []    
            for i in listy:
                if i.startswith("##"):
                    continue
                elif i.startswith('#CHROM'):
                    continue
                else:
                    line1.append([i])
            
            finhead = (headery[0][1:]).split("\t")      
            listdict = []
            for linelist in line1:
                for eachline in linelist:
                    listdict.append(create_dict_from_line(finhead, eachline))
            #print(type(listdict))        
            return listdict
    
    data = read_vcf_file(file)
    def extract_info_field(data):
    
        #print(data[1])
    
        infofield = []
        for i in data:
            infofield.append(i["INFO"])
        return infofield
    ################
    infofield = extract_info_field(data)
    #print(infofield[0])
    ########################
    def create_dictionary_of_info_field_values(data):
    
        infofieldval = []
        for i in data:
            infofieldval.append(i.split(";"))
    
        for i in infofieldval:
            count = 0
            for y in i:
            #print(type(y))
                i[count] = y.split("=")
                count +=1 
        
    #print(infofieldval)
        listnewdict = []
        for i in infofieldval:
            keys = []
            values = []
            for y in i:
                try:
                    values.append(y[1])
                    keys.append(y[0])
                
                except:
                    continue
            newdict = (dict(zip(keys,values)))   
            for key,value in newdict.items():
                newdict[key] = value.replace("\\x3d","=")
        
            for key,value in newdict.items():
                newdict[key] = value.replace("\\x3b",",")
    
    
            for key,value in newdict.items():
                if newdict[key] == '.':
                    newdict[key] = []
      
            for key,value in newdict.items():
                if newdict[key] == []:
                    continue;
                else:
                    newdict[key] = [value]  
            listnewdict.append(newdict)
    #print(newdict)
        if len(data) == 1:
            return newdict
        else:
            return listnewdict
    #########################
    ans = create_dictionary_of_info_field_values(infofield)
   
    
    #########################
    keys_save_value = [
        'FATHMM_pred',
        'LRT_pred',
        'MetaLR_pred',
        'MetaSVM_pred',
        'MutationAssessor_pred',
        'MutationTaster_pred',
        'PROVEAN_pred',
        'Polyphen2_HDIV_pred',
        'Polyphen2_HVAR_pred',
        'SIFT_pred',
        'fathmm-MKL_coding_pred',
    ]
    dictodi = {i : [] for i in keys_save_value}
    
    for z in ans:
        for i,j in z.items():
            for q,r in dictodi.items():
                if i == q:
                    if len(j) != 0:
                        if j[0] not in r:
                        #print(len(j))
                            r.append(j[0].strip())
                        
    
        
        
    return dictodi
            
            
    #print(p)
    # YOUR CODE HERE
    #raise NotImplementedError()


In [4]:

## Can take 30 seconds to run!
expected_solution = {'SIFT_pred': ['D', 'T'], 'Polyphen2_HDIV_pred': ['D', 'B', 'P'], 'Polyphen2_HVAR_pred': ['D', 'B', 'P'], 'LRT_pred': ['D', 'N', 'U'], 'MutationTaster_pred': ['D', 'P', 'N', 'A'], 'MutationAssessor_pred': ['H', 'N', 'L', 'M'], 'FATHMM_pred': ['T', 'D'], 'PROVEAN_pred': ['D', 'N'], 'MetaSVM_pred': ['D', 'T'], 'MetaLR_pred': ['D', 'T'], 'fathmm-MKL_coding_pred': ['D', 'N']}
filename = 'test_4families_annovar.vcf.gz'
predictor_values = get_predictor_values(filename)
assert predictor_values == expected_solution



# Part 2 (No points)

You will now create 13 tables
Tables 1-11 will be for :

FATHMM_pred
LRT_pred 
MetaLR_pred
MetaSVM_pred
MutationAssessor_pred
MutationTaster_pred
PROVEAN_pred
Polyphen2_HDIV_pred
Polyphen2_HVAR_pred
SIFT_pred
fathmm_MKL_coding_pred ## NOTE: I have replaced the dash with an underscore. 

The first column for each of these tables will be the name of the field + ID, e.g., FATHMM_predID. This
column will be of type not null primary key. The second column will be called `prediction` and will be of
type TEXT not null. The prediction values will be values you extracted for each of the fields in the previous step. 
For example, 'FATHMM_pred' has two prediction values 'T' and 'D'. Name the tables after their field name, e.g.,  
call table that will contains values of FATHMM_pred `FATHMM_pred`.  Make sure to sort the values, meaning, D should 
be inserted before T. 


Table 12 will be the Variants table. 
The first column will be VariantID. 

The other columns will be:
CHROM   
POS 
ID  
REF 
ALT 
QUAL    
FILTER
thousandg2015aug_all ##NOTE: a column name cannot start with a number, so you have to rename!
ExAC_ALL


Table 12 will have 11 more columns that relate to each of prediction table. Consider writing a utility function
to fetch the primary key for a given prediction from each of the prediction table. Name each of the 
column should be the name of predictor + ID, e.g., FATHMM_predID. 


You have already deteremined their data type, so use that to set their data type. 
For integer use INTEGER. 
For float use REAL. 
For string use TEXT. 

Table 13  will be called PredictionStats. The first column will be PredictorStatsID INTEGER NOT NULL PRIMARY KEY. 
The second column will be VariantID. The third column will be PredictorName. The fourth column will be PredictorValue. 
This is not a normalized table!

The prediction value will be a float value mapped from the prediction text.  Use the following information to 
assign values: 
REF: https://brb.nci.nih.gov/seqtools/colexpanno.html#dbnsfp

FATHMM_pred
- T = 0
- D = 1

LRT_pred 
- D = 1
- N = 0
- U = 0

MetaLR_pred
- T = 0
- D = 1

MetaSVM_pred
- T = 0
- D = 1

MutationAssessor_pred
- H = 1
- N = 0
- L = 0.25
- M = 0.5

MutationTaster_pred
- D = 1
- P = 0
- N = 0
- A = 1

PROVEAN_pred
- D = 1
- N = 0

Polyphen2_HDIV_pred
- D = 1
- B = 0
- P = 0.5

Polyphen2_HVAR_pred
- D = 1
- B = 0
- P = 0.5

SIFT_pred
- D = 1
- T = 0

fathmm-MKL_coding_pred
- D = 1
- N = 0


```

prediction_mapping = {
    'FATHMM_pred': {'T': 0, 'D': 1},
    'MetaLR_pred': {'T': 0, 'D': 1},
    'MetaSVM_pred': {'T': 0, 'D': 1},
    'SIFT_pred': {'T': 0, 'D': 1},
    'fathmm_MKL_coding_pred': {'D': 1, 'N': 0},
    'LRT_pred': {'U': 0, 'N': 0, 'D': 1},
    'MutationAssessor_pred': {'H': 1, 'N': 0, 'L': 0.25, 'M': 0.5},  
    'MutationTaster_pred': {'D': 1, 'P': 0, 'N': 0, 'A': 1},
    'PROVEAN_pred': {'D': 1, 'N': 0},
    'Polyphen2_HDIV_pred': {'D': 1, 'B': 0, 'P': 0.5},
    'Polyphen2_HVAR_pred': {'D': 1, 'B': 0, 'P': 0.5},
}


```


The idea is that 11 predictors have been used to annotate all the variants in the file. This table combines all that information in one table. 
By grouping the table based on variantid and summing the prediction values mapped from the prediction text, you can find which variants have a consensus on their being deterimental. 

IMPORTANT: instead of fathmm-MKL_coding_pred use fathmm_MKL_coding_pred everywhere. 
HINT: You have to commit the changes you make to the table, otherwise your changes will not be saved. Consider wrapping all changes inside `with conn:`




In [5]:
# Creating lab2.db 
db_file = 'lab2.db'
conn = create_connection(db_file, delete_db=True)
conn.close()


In [6]:
def create_tables_1_11(db_file):
    """
    Create tables 1 to 11.
    
    INPUT: db_file -- Name of database file -- eg. lab2.db
    
    """
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    cursor.execute("PRAGMA foreign_keys = 1")
    
    with conn:
        
        create_table_fathmm_pred = """CREATE TABLE FATHMM_pred(FATHMM_predID INTEGER NOT NULL PRIMARY KEY, PREDICTION TEXT NOT NULL);
        INSERT INTO FATHMM_pred VALUES(1, 'D');
        INSERT INTO FATHMM_pred VALUES(2, 'T');"""
        
        create_table_lrt_pred = """CREATE TABLE LRT_pred(LRT_predID INTEGER NOT NULL PRIMARY KEY, PREDICTION TEXT NOT NULL);
        INSERT INTO LRT_pred VALUES(1, 'D');
        INSERT INTO LRT_pred VALUES(2, 'N');
        INSERT INTO LRT_pred VALUES(3, 'U');"""
        
        create_table_metalr_pred = """CREATE TABLE MetaLR_pred(MetaLR_predID INTEGER NOT NULL PRIMARY KEY, PREDICTION TEXT NOT NULL);
        INSERT INTO MetaLR_pred VALUES(1, 'D');
        INSERT INTO MetaLR_pred VALUES(2, 'T');       
        """
        create_table_metasvm = """CREATE TABLE MetaSVM_pred(MetaSVM_predID INTEGER NOT NULL PRIMARY KEY, PREDICTION TEXT NOT NULL);
        INSERT INTO MetaSVM_pred VALUES(1, 'D');
        INSERT INTO MetaSVM_pred VALUES(2, 'T');
        """
        create_table_mutationassesor = """CREATE TABLE MutationAssessor_pred(MutationAssessor_predID INTEGER NOT NULL PRIMARY KEY, PREDICTION TEXT NOT NULL);
        INSERT INTO MutationAssessor_pred VALUES(1, 'H');
        INSERT INTO MutationAssessor_pred VALUES(2, 'L');
        INSERT INTO MutationAssessor_pred VALUES(3, 'M');
        INSERT INTO MutationAssessor_pred VALUES(4, 'N');
        """
        create_table_mutationtaster = """CREATE TABLE MutationTaster_pred(MutationTaster_predID INTEGER NOT NULL PRIMARY KEY, PREDICTION TEXT NOT NULL);
        INSERT INTO MutationTaster_pred VALUES(1, 'A');
        INSERT INTO MutationTaster_pred VALUES(2, 'D');
        INSERT INTO MutationTaster_pred VALUES(3, 'N');
        INSERT INTO MutationTaster_pred VALUES(4, 'P');   
        """
        create_table_provean = """CREATE TABLE PROVEAN_pred(PROVEAN_predID INTEGER NOT NULL PRIMARY KEY, PREDICTION TEXT NOT NULL);
        INSERT INTO PROVEAN_pred VALUES(1, 'D');
        INSERT INTO PROVEAN_pred VALUES(2, 'N');      
        """
        
        create_table_polyphen = """CREATE TABLE Polyphen2_HDIV_pred(Polyphen2_HDIV_predID INTEGER NOT NULL PRIMARY KEY, PREDICTION TEXT NOT NULL);
        INSERT INTO Polyphen2_HDIV_pred VALUES(1, 'B');
        INSERT INTO Polyphen2_HDIV_pred VALUES(2, 'D');
        INSERT INTO Polyphen2_HDIV_pred VALUES(3, 'P');
        """
    
        create_table_polyphen_hvar = """CREATE TABLE Polyphen2_HVAR_pred(Polyphen2_HVAR_predID INTEGER NOT NULL PRIMARY KEY, PREDICTION TEXT NOT NULL);
        INSERT INTO Polyphen2_HVAR_pred VALUES(1, 'B');
        INSERT INTO Polyphen2_HVAR_pred VALUES(2, 'D');
        INSERT INTO Polyphen2_HVAR_pred VALUES(3, 'P');       
        """
    
        create_table_sift = """CREATE TABLE SIFT_pred(SIFT_predID INTEGER NOT NULL PRIMARY KEY, PREDICTION TEXT NOT NULL);
        INSERT INTO SIFT_pred VALUES(1, 'D');
        INSERT INTO SIFT_pred VALUES(2, 'T');     
        """
        
        create_table_fathmmmkl = """CREATE TABLE fathmm_MKL_coding_pred(fathmm_MKL_coding_predID INTEGER NOT NULL PRIMARY KEY, PREDICTION TEXT NOT NULL);
        INSERT INTO fathmm_MKL_coding_pred VALUES(1, 'D');
        INSERT INTO fathmm_MKL_coding_pred VALUES(2, 'N');
        """
    
        cursor.executescript(create_table_fathmm_pred)
        cursor.executescript(create_table_lrt_pred)
        cursor.executescript(create_table_metalr_pred)
        cursor.executescript(create_table_metasvm)
        cursor.executescript(create_table_mutationassesor)
        cursor.executescript(create_table_mutationtaster)
        cursor.executescript(create_table_provean)
        cursor.executescript(create_table_polyphen)
        cursor.executescript(create_table_polyphen_hvar)
        cursor.executescript(create_table_sift)
        cursor.executescript(create_table_fathmmmkl)
    
    
    #cursor.executescript
    #conn.commit()
    #conn.close()

    # YOUR CODE HERE
    #raise NotImplementedError()

db_file = 'lab2.db'    
create_tables_1_11(db_file)

# Part 3 (10 pts)

Write a function that returns a dictionary that maps for a given predictor its letter prediction to foreign key value. 
Conver the dash in 'fathmm-MKL_coding_pred' to an underscore. 


In [7]:
def get_predictor_value_to_fk_map(db_file):
    """
    See part 3 description 
    
    INPUT: db_file -- Name of database file -- eg. lab2.db
    
    """
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master where type = 'table'")
    tables = cursor.fetchall()
    
    listy = []
    for i in range(len(tables)):
        listy.append(tables[i][0])
    
    
    dict1 = {}
    for i in listy:
        #print(name[0])
        cursor.execute(f"SELECT * FROM {i}")
        data = cursor.fetchall()
        keys = []
        values = []
        for j in range(len(data)):
            keys.append(data[j][1])
            values.append(data[j][0])
        dict1[i] = dict(zip(keys,values))
        
    
    return dict1
    #conn.commit()
    #conn.close()
    

    
        
    
    # YOUR CODE HERE
    #raise NotImplementedError()


In [8]:
expected_solution = {
    'FATHMM_pred': {'D': 1, 'T': 2}, 
    'LRT_pred': {'D': 1, 'N': 2, 'U': 3}, 
    'MetaLR_pred': {'D': 1, 'T': 2}, 
    'MetaSVM_pred': {'D': 1, 'T': 2}, 
    'MutationAssessor_pred': {'H': 1, 'L': 2, 'M': 3, 'N': 4}, 
    'MutationTaster_pred': {'A': 1, 'D': 2, 'N': 3, 'P': 4}, 
    'PROVEAN_pred': {'D': 1, 'N': 2}, 
    'Polyphen2_HDIV_pred': {'B': 1, 'D': 2, 'P': 3}, 
    'Polyphen2_HVAR_pred': {'B': 1, 'D': 2, 'P': 3}, 
    'SIFT_pred': {'D': 1, 'T': 2}, 
    'fathmm_MKL_coding_pred': {'D': 1, 'N': 2}}

db_file = 'lab2.db'
predictor_fk_map = get_predictor_value_to_fk_map(db_file)
assert predictor_fk_map == expected_solution



# Part 4 (No Points)
Create table 12 or the variants table. See description above


In [9]:
def create_variants_table(db_file):
    """
    Part 4
    """
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    cursor.execute("PRAGMA foreign_keys = 1")
    
    
    create_table_variants = """CREATE TABLE Variants(
    VariantID INTEGER NOT NULL PRIMARY KEY, 
    CHROM TEXT,
    POS INTEGER,
    ID  TEXT,
    REF TEXT,
    ALT TEXT,
    QUAL REAL,
    FILTER TEXT,
    THOUSANDG2015AUG_ALL REAL, 
    ExAC_ALL REAL,
    FATHMM_predID INTEGER,
    LRT_predID INTEGER,
    MetaLR_predID INTEGER,
    MetaSVM_predID INTEGER,
    MutationAssessor_predID INTEGER,
    MutationTaster_predID INTEGER,
    PROVEAN_predID INTEGER,
    Polyphen2_HDIV_predID INTEGER,
    Polyphen2_HVAR_predID INTEGER,
    SIFT_predID INTEGER,
    fathmm_MKL_coding_predID INTEGER)
    """
    
    with conn:
        create_table(conn, create_table_variants)
    
    #conn.commit()
    #conn.close()
    # YOUR CODE HERE
    #raise NotImplementedError()

# create table
db_file = 'lab2.db'
create_variants_table(db_file)


# Part 5 (No Points)

Create table 13 -- or the prediction stats table. See description above. 


In [10]:
def create_predictionstats_table(db_file):
    """
    Part 5   
    """
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    cursor.execute("PRAGMA foreign_keys = 1")
    
    create_table_predict = """CREATE TABLE PredictionStats(
    PredictorStatsID INTEGER NOT NULL PRIMARY KEY,
    VariantID INTEGER,
    PredictorName TEXT,
    PredictorValue REAL);"""
    
    with conn:
        create_table(conn, create_table_predict)
    
    #conn.commit()
    #conn.close()

    
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    
db_file = 'lab2.db'
create_predictionstats_table(db_file)

# Part 6 (10 Points)

Write a function to pull the following info fields given the whole info field. 
```
values_to_pull = [
        '1000g2015aug_all',
        'ExAC_ALL',
        'FATHMM_pred',
        'LRT_pred',
        'MetaLR_pred',
        'MetaSVM_pred',
        'MutationAssessor_pred',
        'MutationTaster_pred',
        'PROVEAN_pred',
        'Polyphen2_HDIV_pred',
        'Polyphen2_HVAR_pred',
        'SIFT_pred',
        'fathmm-MKL_coding_pred',
    ]
```


In [11]:
sample_info_input = "AC=2;AF=0.333;AN=6;BaseQRankSum=2.23;ClippingRankSum=0;DP=131;ExcessHet=3.9794;FS=2.831;MLEAC=2;MLEAF=0.333;MQ=60;MQRankSum=0;QD=12.06;ReadPosRankSum=-0.293;SOR=0.592;VQSLOD=21.79;culprit=MQ;DB;POSITIVE_TRAIN_SITE;ANNOVAR_DATE=2018-04-16;Func.refGene=exonic;Gene.refGene=MAST2;GeneDetail.refGene=.;ExonicFunc.refGene=nonsynonymous_SNV;AAChange.refGene=MAST2:NM_015112:exon29:c.G3910A:p.V1304M;Func.ensGene=exonic;Gene.ensGene=ENSG00000086015;GeneDetail.ensGene=.;ExonicFunc.ensGene=nonsynonymous_SNV;AAChange.ensGene=ENSG00000086015:ENST00000361297:exon29:c.G3910A:p.V1304M;cytoBand=1p34.1;gwasCatalog=.;tfbsConsSites=.;wgRna=.;targetScanS=.;Gene_symbol=.;OXPHOS_Complex=.;Ensembl_Gene_ID=.;Ensembl_Protein_ID=.;Uniprot_Name=.;Uniprot_ID=.;NCBI_Gene_ID=.;NCBI_Protein_ID=.;Gene_pos=.;AA_pos=.;AA_sub=.;Codon_sub=.;dbSNP_ID=.;PhyloP_46V=.;PhastCons_46V=.;PhyloP_100V=.;PhastCons_100V=.;SiteVar=.;PolyPhen2_prediction=.;PolyPhen2_score=.;SIFT_prediction=.;SIFT_score=.;FatHmm_prediction=.;FatHmm_score=.;PROVEAN_prediction=.;PROVEAN_score=.;MutAss_prediction=.;MutAss_score=.;EFIN_Swiss_Prot_Score=.;EFIN_Swiss_Prot_Prediction=.;EFIN_HumDiv_Score=.;EFIN_HumDiv_Prediction=.;CADD_score=.;CADD_Phred_score=.;CADD_prediction=.;Carol_prediction=.;Carol_score=.;Condel_score=.;Condel_pred=.;COVEC_WMV=.;COVEC_WMV_prediction=.;PolyPhen2_score_transf=.;PolyPhen2_pred_transf=.;SIFT_score_transf=.;SIFT_pred_transf=.;MutAss_score_transf=.;MutAss_pred_transf=.;Perc_coevo_Sites=.;Mean_MI_score=.;COSMIC_ID=.;Tumor_site=.;Examined_samples=.;Mutation_frequency=.;US=.;Status=.;Associated_disease=.;Presence_in_TD=.;Class_predicted=.;Prob_N=.;Prob_P=.;SIFT_score=0.034;SIFT_converted_rankscore=0.440;SIFT_pred=D;Polyphen2_HDIV_score=0.951;Polyphen2_HDIV_rankscore=0.520;Polyphen2_HDIV_pred=P;Polyphen2_HVAR_score=0.514;Polyphen2_HVAR_rankscore=0.462;Polyphen2_HVAR_pred=P;LRT_score=0.002;LRT_converted_rankscore=0.368;LRT_pred=N;MutationTaster_score=1.000;MutationTaster_converted_rankscore=0.810;MutationTaster_pred=D;MutationAssessor_score=1.67;MutationAssessor_score_rankscore=0.430;MutationAssessor_pred=L;FATHMM_score=1.36;FATHMM_converted_rankscore=0.344;FATHMM_pred=T;PROVEAN_score=-1.4;PROVEAN_converted_rankscore=0.346;PROVEAN_pred=N;VEST3_score=0.158;VEST3_rankscore=0.189;MetaSVM_score=-1.142;MetaSVM_rankscore=0.013;MetaSVM_pred=T;MetaLR_score=0.008;MetaLR_rankscore=0.029;MetaLR_pred=T;M-CAP_score=.;M-CAP_rankscore=.;M-CAP_pred=.;CADD_raw=4.716;CADD_raw_rankscore=0.632;CADD_phred=24.6;DANN_score=0.998;DANN_rankscore=0.927;fathmm-MKL_coding_score=0.900;fathmm-MKL_coding_rankscore=0.506;fathmm-MKL_coding_pred=D;Eigen_coding_or_noncoding=c;Eigen-raw=0.461;Eigen-PC-raw=0.469;GenoCanyon_score=1.000;GenoCanyon_score_rankscore=0.747;integrated_fitCons_score=0.672;integrated_fitCons_score_rankscore=0.522;integrated_confidence_value=0;GERP++_RS=4.22;GERP++_RS_rankscore=0.490;phyloP100way_vertebrate=4.989;phyloP100way_vertebrate_rankscore=0.634;phyloP20way_mammalian=1.047;phyloP20way_mammalian_rankscore=0.674;phastCons100way_vertebrate=1.000;phastCons100way_vertebrate_rankscore=0.715;phastCons20way_mammalian=0.999;phastCons20way_mammalian_rankscore=0.750;SiPhy_29way_logOdds=17.151;SiPhy_29way_logOdds_rankscore=0.866;Interpro_domain=.;GTEx_V6_gene=ENSG00000162415.6;GTEx_V6_tissue=Nerve_Tibial;esp6500siv2_all=0.0560;esp6500siv2_aa=0.0160;esp6500siv2_ea=0.0761;ExAC_ALL=0.0553;ExAC_AFR=0.0140;ExAC_AMR=0.0386;ExAC_EAS=0.0005;ExAC_FIN=0.0798;ExAC_NFE=0.0788;ExAC_OTH=0.0669;ExAC_SAS=0.0145;ExAC_nontcga_ALL=0.0541;ExAC_nontcga_AFR=0.0129;ExAC_nontcga_AMR=0.0379;ExAC_nontcga_EAS=0.0004;ExAC_nontcga_FIN=0.0798;ExAC_nontcga_NFE=0.0802;ExAC_nontcga_OTH=0.0716;ExAC_nontcga_SAS=0.0144;ExAC_nonpsych_ALL=0.0496;ExAC_nonpsych_AFR=0.0140;ExAC_nonpsych_AMR=0.0386;ExAC_nonpsych_EAS=0.0005;ExAC_nonpsych_FIN=0.0763;ExAC_nonpsych_NFE=0.0785;ExAC_nonpsych_OTH=0.0638;ExAC_nonpsych_SAS=0.0145;1000g2015aug_all=0.024361;1000g2015aug_afr=0.0038;1000g2015aug_amr=0.0461;1000g2015aug_eur=0.0795;1000g2015aug_sas=0.0041;CLNALLELEID=.;CLNDN=.;CLNDISDB=.;CLNREVSTAT=.;CLNSIG=.;dbscSNV_ADA_SCORE=.;dbscSNV_RF_SCORE=.;snp138NonFlagged=rs33931638;avsnp150=rs33931638;CADD13_RawScore=4.716301;CADD13_PHRED=24.6;Eigen=0.4614;REVEL=0.098;MCAP=.;Interpro_domain=.;ICGC_Id=.;ICGC_Occurrence=.;gnomAD_genome_ALL=0.0507;gnomAD_genome_AFR=0.0114;gnomAD_genome_AMR=0.0430;gnomAD_genome_ASJ=0.1159;gnomAD_genome_EAS=0;gnomAD_genome_FIN=0.0802;gnomAD_genome_NFE=0.0702;gnomAD_genome_OTH=0.0695;gerp++gt2=4.22;cosmic70=.;InterVar_automated=Benign;PVS1=0;PS1=0;PS2=0;PS3=0;PS4=0;PM1=0;PM2=0;PM3=0;PM4=0;PM5=0;PM6=0;PP1=0;PP2=0;PP3=0;PP4=0;PP5=0;BA1=1;BS1=1;BS2=0;BS3=0;BS4=0;BP1=0;BP2=0;BP3=0;BP4=0;BP5=0;BP6=0;BP7=0;Kaviar_AF=0.0552127;Kaviar_AC=8536;Kaviar_AN=154602;ALLELE_END"
values_to_pull = [
        '1000g2015aug_all',
        'ExAC_ALL',
        'FATHMM_pred',
        'LRT_pred',
        'MetaLR_pred',
        'MetaSVM_pred',
        'MutationAssessor_pred',
        'MutationTaster_pred',
        'PROVEAN_pred',
        'Polyphen2_HDIV_pred',
        'Polyphen2_HVAR_pred',
        'SIFT_pred',
        'fathmm-MKL_coding_pred',
    ]
def pull_info_values(info):
    """
    See part 6 description
    """
    sample = info.split(";")
    
    sampley = [i.split("=") for i in sample]
        
    list1 = []
    list2 = []
    for i in sampley:
        for j in values_to_pull:
            if i[0] == j:
                if i[0] == '1000g2015aug_all':
                    list1.append('thousandg2015aug_all')
                    list2.append(i[1])
                    
                if i[0] == 'fathmm-MKL_coding_pred':
                    list1.append('fathmm_MKL_coding_pred')
                    list2.append(i[1])
                    
                else:
                    if i[0] != '1000g2015aug_all':
                        if i[0] != 'fathmm-MKL_coding_pred':
                            list1.append(i[0].strip())
                            list2.append(i[1].strip())
                
    findict = dict(zip(list1,list2))
    #print(findict)
    return findict
    # YOUR CODE HERE
    #raise NotImplementedError()



In [12]:
sample_info_input = "AC=2;AF=0.333;AN=6;BaseQRankSum=2.23;ClippingRankSum=0;DP=131;ExcessHet=3.9794;FS=2.831;MLEAC=2;MLEAF=0.333;MQ=60;MQRankSum=0;QD=12.06;ReadPosRankSum=-0.293;SOR=0.592;VQSLOD=21.79;culprit=MQ;DB;POSITIVE_TRAIN_SITE;ANNOVAR_DATE=2018-04-16;Func.refGene=exonic;Gene.refGene=MAST2;GeneDetail.refGene=.;ExonicFunc.refGene=nonsynonymous_SNV;AAChange.refGene=MAST2:NM_015112:exon29:c.G3910A:p.V1304M;Func.ensGene=exonic;Gene.ensGene=ENSG00000086015;GeneDetail.ensGene=.;ExonicFunc.ensGene=nonsynonymous_SNV;AAChange.ensGene=ENSG00000086015:ENST00000361297:exon29:c.G3910A:p.V1304M;cytoBand=1p34.1;gwasCatalog=.;tfbsConsSites=.;wgRna=.;targetScanS=.;Gene_symbol=.;OXPHOS_Complex=.;Ensembl_Gene_ID=.;Ensembl_Protein_ID=.;Uniprot_Name=.;Uniprot_ID=.;NCBI_Gene_ID=.;NCBI_Protein_ID=.;Gene_pos=.;AA_pos=.;AA_sub=.;Codon_sub=.;dbSNP_ID=.;PhyloP_46V=.;PhastCons_46V=.;PhyloP_100V=.;PhastCons_100V=.;SiteVar=.;PolyPhen2_prediction=.;PolyPhen2_score=.;SIFT_prediction=.;SIFT_score=.;FatHmm_prediction=.;FatHmm_score=.;PROVEAN_prediction=.;PROVEAN_score=.;MutAss_prediction=.;MutAss_score=.;EFIN_Swiss_Prot_Score=.;EFIN_Swiss_Prot_Prediction=.;EFIN_HumDiv_Score=.;EFIN_HumDiv_Prediction=.;CADD_score=.;CADD_Phred_score=.;CADD_prediction=.;Carol_prediction=.;Carol_score=.;Condel_score=.;Condel_pred=.;COVEC_WMV=.;COVEC_WMV_prediction=.;PolyPhen2_score_transf=.;PolyPhen2_pred_transf=.;SIFT_score_transf=.;SIFT_pred_transf=.;MutAss_score_transf=.;MutAss_pred_transf=.;Perc_coevo_Sites=.;Mean_MI_score=.;COSMIC_ID=.;Tumor_site=.;Examined_samples=.;Mutation_frequency=.;US=.;Status=.;Associated_disease=.;Presence_in_TD=.;Class_predicted=.;Prob_N=.;Prob_P=.;SIFT_score=0.034;SIFT_converted_rankscore=0.440;SIFT_pred=D;Polyphen2_HDIV_score=0.951;Polyphen2_HDIV_rankscore=0.520;Polyphen2_HDIV_pred=P;Polyphen2_HVAR_score=0.514;Polyphen2_HVAR_rankscore=0.462;Polyphen2_HVAR_pred=P;LRT_score=0.002;LRT_converted_rankscore=0.368;LRT_pred=N;MutationTaster_score=1.000;MutationTaster_converted_rankscore=0.810;MutationTaster_pred=D;MutationAssessor_score=1.67;MutationAssessor_score_rankscore=0.430;MutationAssessor_pred=L;FATHMM_score=1.36;FATHMM_converted_rankscore=0.344;FATHMM_pred=T;PROVEAN_score=-1.4;PROVEAN_converted_rankscore=0.346;PROVEAN_pred=N;VEST3_score=0.158;VEST3_rankscore=0.189;MetaSVM_score=-1.142;MetaSVM_rankscore=0.013;MetaSVM_pred=T;MetaLR_score=0.008;MetaLR_rankscore=0.029;MetaLR_pred=T;M-CAP_score=.;M-CAP_rankscore=.;M-CAP_pred=.;CADD_raw=4.716;CADD_raw_rankscore=0.632;CADD_phred=24.6;DANN_score=0.998;DANN_rankscore=0.927;fathmm-MKL_coding_score=0.900;fathmm-MKL_coding_rankscore=0.506;fathmm-MKL_coding_pred=D;Eigen_coding_or_noncoding=c;Eigen-raw=0.461;Eigen-PC-raw=0.469;GenoCanyon_score=1.000;GenoCanyon_score_rankscore=0.747;integrated_fitCons_score=0.672;integrated_fitCons_score_rankscore=0.522;integrated_confidence_value=0;GERP++_RS=4.22;GERP++_RS_rankscore=0.490;phyloP100way_vertebrate=4.989;phyloP100way_vertebrate_rankscore=0.634;phyloP20way_mammalian=1.047;phyloP20way_mammalian_rankscore=0.674;phastCons100way_vertebrate=1.000;phastCons100way_vertebrate_rankscore=0.715;phastCons20way_mammalian=0.999;phastCons20way_mammalian_rankscore=0.750;SiPhy_29way_logOdds=17.151;SiPhy_29way_logOdds_rankscore=0.866;Interpro_domain=.;GTEx_V6_gene=ENSG00000162415.6;GTEx_V6_tissue=Nerve_Tibial;esp6500siv2_all=0.0560;esp6500siv2_aa=0.0160;esp6500siv2_ea=0.0761;ExAC_ALL=0.0553;ExAC_AFR=0.0140;ExAC_AMR=0.0386;ExAC_EAS=0.0005;ExAC_FIN=0.0798;ExAC_NFE=0.0788;ExAC_OTH=0.0669;ExAC_SAS=0.0145;ExAC_nontcga_ALL=0.0541;ExAC_nontcga_AFR=0.0129;ExAC_nontcga_AMR=0.0379;ExAC_nontcga_EAS=0.0004;ExAC_nontcga_FIN=0.0798;ExAC_nontcga_NFE=0.0802;ExAC_nontcga_OTH=0.0716;ExAC_nontcga_SAS=0.0144;ExAC_nonpsych_ALL=0.0496;ExAC_nonpsych_AFR=0.0140;ExAC_nonpsych_AMR=0.0386;ExAC_nonpsych_EAS=0.0005;ExAC_nonpsych_FIN=0.0763;ExAC_nonpsych_NFE=0.0785;ExAC_nonpsych_OTH=0.0638;ExAC_nonpsych_SAS=0.0145;1000g2015aug_all=0.024361;1000g2015aug_afr=0.0038;1000g2015aug_amr=0.0461;1000g2015aug_eur=0.0795;1000g2015aug_sas=0.0041;CLNALLELEID=.;CLNDN=.;CLNDISDB=.;CLNREVSTAT=.;CLNSIG=.;dbscSNV_ADA_SCORE=.;dbscSNV_RF_SCORE=.;snp138NonFlagged=rs33931638;avsnp150=rs33931638;CADD13_RawScore=4.716301;CADD13_PHRED=24.6;Eigen=0.4614;REVEL=0.098;MCAP=.;Interpro_domain=.;ICGC_Id=.;ICGC_Occurrence=.;gnomAD_genome_ALL=0.0507;gnomAD_genome_AFR=0.0114;gnomAD_genome_AMR=0.0430;gnomAD_genome_ASJ=0.1159;gnomAD_genome_EAS=0;gnomAD_genome_FIN=0.0802;gnomAD_genome_NFE=0.0702;gnomAD_genome_OTH=0.0695;gerp++gt2=4.22;cosmic70=.;InterVar_automated=Benign;PVS1=0;PS1=0;PS2=0;PS3=0;PS4=0;PM1=0;PM2=0;PM3=0;PM4=0;PM5=0;PM6=0;PP1=0;PP2=0;PP3=0;PP4=0;PP5=0;BA1=1;BS1=1;BS2=0;BS3=0;BS4=0;BP1=0;BP2=0;BP3=0;BP4=0;BP5=0;BP6=0;BP7=0;Kaviar_AF=0.0552127;Kaviar_AC=8536;Kaviar_AN=154602;ALLELE_END"

expected_solution = {
    'thousandg2015aug_all': '0.024361', 
    'ExAC_ALL': '0.0553', 
    'SIFT_pred': 'D', 
    'Polyphen2_HDIV_pred': 'P', 
    'Polyphen2_HVAR_pred': 'P', 
    'LRT_pred': 'N',
    'MutationTaster_pred': 'D', 
    'MutationAssessor_pred': 'L', 
    'FATHMM_pred': 'T', 
    'PROVEAN_pred': 'N', 
    'MetaSVM_pred': 'T', 
    'MetaLR_pred': 'T', 
    'fathmm_MKL_coding_pred': 'D'
}


solution  = pull_info_values(sample_info_input)
assert solution == expected_solution


# Part 7 (10 points)

Remember that to insert a record in SQLite, you have to use `cur.execute(sql, values)`, where `sql` is the insert statement and `values` is a list/tuple of values that will be substituted into the `sql` string wherever there is a question mark. 

Write a function that takes in as input: CHROM, POS, ID, REF, ALT, QUAL, FILTER, info_values and returns a list with the values in the following order:
```
CHROM 
POS
ID
REF
ALT
QUAL
FILTER
thousandg2015aug_all # 1000 has been replaced by the text thousand
ExAC_ALL
FATHMM_pred
LRT_pred
MetaLR_pred
MetaSVM_pred
MutationAssessor_pred
MutationTaster_pred
PROVEAN_pred
Polyphen2_HDIV_pred
Polyphen2_HVAR_pred
SIFT_pred
fathmm_MKL_coding_pred # note that the dash has been replaced by underscore
```

The info_values dictionary contains the predictor values. Use `None` for any empty/missing value for info fields, thousandg2015aug_all, and ExAC_ALL. 



In [13]:
def build_values_list(CHROM, POS, ID, REF, ALT, QUAL, FILTER, info_values):
    """
    See part 7 description 
    
    """
    dicti = predictor_fk_map
    #print(dicti)
    
    listy = [CHROM,POS,ID,REF,ALT,QUAL,FILTER]

    for k,v in info_values.items():
        try:
            float(v)
        except:
            if '.' in v:
                info_values[k] = None


    for k,v in info_values.items():
        
        if k == 'thousandg2015aug_all':
            listy.append(info_values[k])
        
        elif k == 'ExAC_ALL':
            listy.append(info_values[k])

    if 'thousandg2015aug_all' not in info_values.keys(): 
        listy.append(None)

    if 'ExAC_ALL' not in info_values.keys():
        listy.append(None)

    for k,v in dicti.items():
        
        if k in info_values.keys():
            value = info_values[k]
        
            if value in v.keys():
                listy.append(v[value])
            else:
                listy.append(value)
    #print(listy)
    return listy
    # YOUR CODE HERE
    #raise NotImplementedError()



In [14]:
CHROM, POS, ID, REF, ALT, QUAL, FILTER = (7, 87837848, '.', 'C', 'A', 418.25, 'PASS') 
info_values = {'SIFT_pred': 'D', 'Polyphen2_HDIV_pred': 'D', 'Polyphen2_HVAR_pred': 'D', 'LRT_pred': 'D', 'MutationTaster_pred': 'D', 'MutationAssessor_pred': 'H', 'FATHMM_pred': 'T', 'PROVEAN_pred': 'D', 'MetaSVM_pred': 'D', 'MetaLR_pred': 'D', 'fathmm_MKL_coding_pred': 'D'}

results = build_values_list(CHROM, POS, ID, REF, ALT, QUAL, FILTER, info_values)
expected_results = [7, 87837848, '.', 'C', 'A', 418.25, 'PASS', None, None, 2, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1]
assert results == expected_results


# Part 8 (No Points)

Create a function that takes the database `conn` and `values` from the `build_values_list` function to insert a variant record. 

IMPORTANT: Function should return the id of the row inserted, which will be VariantId


In [15]:
def insert_variant(conn, values):
    """
    See description Part 8
    """
    conn= create_connection(conn)
    cursor = conn.cursor()
    
    sqlstatement = """INSERT INTO VARIANTS (CHROM, POS , ID , REF , ALT , QUAL , FILTER , THOUSANDG2015AUG_ALL , ExAC_ALL, FATHMM_predID, LRT_predID,  MetaLR_predID, MetaSVM_predID, MutationAssessor_predID, MutationTaster_predID, PROVEAN_predID, Polyphen2_HDIV_predID, Polyphen2_HVAR_predID, SIFT_predID, fathmm_MKL_coding_predID) VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"""
    
    
    #conn.commit()
    
    with conn:
        cursor.execute(sqlstatement, values)
        #cursor.execute("select max(variantid) from variants")
        
        #ans = cursor.fetchall()
    #print(cursor.fetchall())
    #conn.commit()
    #conn.close()
    
    #print(ans[0][0])
    #print(cursor.lastrowid)
    return cursor.lastrowid
    
    # YOUR CODE HERE
    #raise NotImplementedError()

# Part 9 (No Points)

Create a function that takes the database `conn` and `values` which is the tuple `(VariantId, PredictorName, PredictorValue)` Where `VariantId` will be the return value from the `insert_variant` function,  `PredictorName` is the name of the predictor, and `PredictorValue` is the mapping of the text value to a numeric value. 


In [16]:
def insert_predictionstat(conn, values):
    """
    See description in part 9
    """
    conn= create_connection(conn)
    
    cursor = conn.cursor()
    
    sqlstatement = "INSERT INTO PredictionStats (VariantID, PredictorName, PredictorValue) VALUES (?, ?, ?)"
    
    with conn:
        cursor.execute(sqlstatement, values)
    
    #conn.commit()
    
    #conn.close()

    
    # YOUR CODE HERE
    #raise NotImplementedError()


# Part 10 (No Points)

Write a function to insert records into both the variants and predictor_stats table. 
Hint:
1) Open connection to database  
2) Read file one line at a time using gzip read
3) Extract CHROM, POS, ID, REF, ALT, QUAL, FILTER, INFO
4) Use the pull_info_values function
5) Use build_values_list 
6) Use insert_variant -- save variant_id
7) Use insert_predictionstat -- insert each predictor at a time and remember to use `prediction_mapping` mapping. 




```

prediction_mapping = {
    'FATHMM_pred': {'T': 0, 'D': 1},
    'MetaLR_pred': {'T': 0, 'D': 1},
    'MetaSVM_pred': {'T': 0, 'D': 1},
    'SIFT_pred': {'T': 0, 'D': 1},
    'fathmm_MKL_coding_pred': {'D': 1, 'N': 0},
    'LRT_pred': {'U': 0, 'N': 0, 'D': 1},
    'MutationAssessor_pred': {'H': 1, 'N': 0, 'L': 0.25, 'M': 0.5},  
    'MutationTaster_pred': {'D': 1, 'P': 0, 'N': 0, 'A': 1},
    'PROVEAN_pred': {'D': 1, 'N': 0},
    'Polyphen2_HDIV_pred': {'D': 1, 'B': 0, 'P': 0.5},
    'Polyphen2_HVAR_pred': {'D': 1, 'B': 0, 'P': 0.5},
}


```


In [17]:
prediction_mapping = {
    'FATHMM_pred': {'T': 0, 'D': 1},
    'MetaLR_pred': {'T': 0, 'D': 1},
    'MetaSVM_pred': {'T': 0, 'D': 1},
    'SIFT_pred': {'T': 0, 'D': 1},
    'fathmm_MKL_coding_pred': {'D': 1, 'N': 0},
    'LRT_pred': {'U': 0, 'N': 0, 'D': 1},
    'MutationAssessor_pred': {'H': 1, 'N': 0, 'L': 0.25, 'M': 0.5},  
    'MutationTaster_pred': {'D': 1, 'P': 0, 'N': 0, 'A': 1},
    'PROVEAN_pred': {'D': 1, 'N': 0},
    'Polyphen2_HDIV_pred': {'D': 1, 'B': 0, 'P': 0.5},
    'Polyphen2_HVAR_pred': {'D': 1, 'B': 0, 'P': 0.5},
}

    #with gzip.open(filename, 'rt') as fp:
    #file = list(fp)
    


def populate_variants_predictorstats_tables(db_file, filename):
    """
    See description in part 10
    """
    # YOUR CODE HERE
    
    listy = []
    
    with gzip.open(filename, 'rt') as fp:
        
        for i in fp:
            if i.startswith('#'):
                continue
            else:
                listy.append(i)

    #print(len(listy))

    for j in listy:
        
        CHROM, POS, ID, REF, ALT, QUAL, FILTER = j.split()[0:7]
        
        info = j.split()[7]
        
        info_values = pull_info_values(info)
        
        values = build_values_list(CHROM, POS, ID, REF, ALT, QUAL, FILTER, info_values)
        
        
        conn = create_connection(db_file)
        #print(conn)
        with conn:
            variantid = insert_variant(db_file, values)

    
            for key,value in info_values.items():
            
                if key == 'thousandg2015aug_all' or  key == 'ExAC_ALL':
                    continue
        
                if key in prediction_mapping.keys():
            
                    dictodi = prediction_mapping[key]
                
                    if value in dictodi.keys():
                    
                        name = key
                    
                        valuez = dictodi[value]
                    
                        values = (variantid, name, valuez)
                    
                        insert_predictionstat(db_file, values)

    
    #raise NotImplementedError()
try:
    conn.close()
except:
    pass

db_file = 'lab2.db'
filename = 'test_4families_annovar.vcf.gz'
populate_variants_predictorstats_tables(db_file, filename)

# Part 11 (10 Points)

Write a function that returns the total number of variants



In [18]:
def num_of_total_variants(conn):
    # YOUR CODE HERE
    cursor = conn.cursor()
    cursor.execute("Select count(*) from variants" )
    answer = cursor.fetchall()
    print((answer))
    return answer[0][0]
#     cursor.execute("Select count(*) from Variants" )
    raise NotImplementedError()


In [19]:
db_file = 'lab2.db'
conn = create_connection(db_file)
assert num_of_total_variants(conn) == 50001
conn.close()

[(50001,)]


# Part 12 (10 Points)
Write a function returns the total number of variant predictions -- the count of the predictiostats table. 

In [20]:
def num_of_total_variant_predictions(conn):
    """
    Part 12
    """
    cursor = conn.cursor()
    cursor.execute("Select count(*) from predictionstats" )
    answer = cursor.fetchall()
    
    return (answer[0][0])
    # YOUR CODE HERE
    #raise NotImplementedError()



In [21]:
db_file = 'lab2.db'
conn = create_connection(db_file)
assert num_of_total_variant_predictions(conn) == 1324
conn.close()

# Part 13 (10 Points)
Return the total number of variant predictions that have value greater than zero. Number of values from the predictionstats table that are greater than 0. 

In [22]:
def num_of_total_variant_predictions_with_value_gt_zero(conn):
    """
    See part 13 description
    """
    cursor = conn.cursor()
    
    cursor.execute("select count(predictorvalue) from predictionstats where predictorvalue > 0")
    
    data = cursor.fetchall()
    #print(data)
    return data[0][0]
    
    # YOUR CODE HERE
#     raise NotImplementedError()



In [23]:
db_file = 'lab2.db'
conn = create_connection(db_file)
assert num_of_total_variant_predictions_with_value_gt_zero(conn) == 219
conn.close()

# Part 14 (10 Points)

Write a function that given `CHROM, POS, ID, REF, ALT` returns a variant's info with the following columns (column order is important) :
```
Variants.CHROM,
Variants.POS,
Variants.ID,
Variants.REF,
Variants.ALT,
Variants.QUAL,
Variants.FILTER,
Variants.thousandg2015aug_all,
Variants.ExAC_ALL,
FATHMM_pred.prediction,
LRT_pred.prediction,
MetaLR_pred.prediction,
MetaSVM_pred.prediction,
MutationAssessor_pred.prediction,
MutationTaster_pred.prediction,
PROVEAN_pred.prediction,
Polyphen2_HDIV_pred.prediction,
Polyphen2_HVAR_pred.prediction,
SIFT_pred.prediction,
fathmm_MKL_coding_pred.prediction,
sum(PredictionStats.PredictorValue)
```

For the predictions, return the actual text value. And the last column is the sum of all the mapped prediction scores for a given variant!



In [24]:
def fetch_variant(conn, CHROM, POS, ID, REF, ALT):
    """
    See Part 14 description
    """
    cursor = conn.cursor()
    cursor.execute("PRAGMA foreign_keys = 1")
    values = (CHROM, POS, ID, REF, ALT)
    
    
    try:
        sql_statement = """select V.CHROM,V.POS,V.ID,V.REF,V.ALT,V.QUAL,V.FILTER,V.ExAC_ALL,V.thousandg2015aug_all,A.prediction,
        B.prediction,
        C.prediction,
        D.prediction,
        E.prediction,
        F.prediction,
        G.prediction,
        H.prediction,
        I.prediction,
        J.prediction,
        K.prediction,
        sum(P.PredictorValue)
        from VARIANTS V 
        inner join FATHMM_pred A on V.FATHMM_predID=A.FATHMM_predID 
        inner join LRT_pred B on V.LRT_predID=B.LRT_predID
        inner join MetaLR_pred C on V.MetaLR_predID=C.MetaLR_predID
        inner join MetaSVM_pred D on V.MetaSVM_predID=D.MetaSVM_predID
        inner join MutationAssessor_pred E on V.MutationAssessor_predID=E.MutationAssessor_predID
        inner join MutationTaster_pred F on V.MutationTaster_predID=F.MutationTaster_predID
        inner join PROVEAN_pred G on V.PROVEAN_predID=G.PROVEAN_predID
        inner join Polyphen2_HDIV_pred H on V.Polyphen2_HDIV_predID=H.Polyphen2_HDIV_predID
        inner join Polyphen2_HVAR_pred I on V.Polyphen2_HVAR_predID=I.Polyphen2_HVAR_predID
        inner join SIFT_pred J on V.SIFT_predID=J.SIFT_predID
        inner join fathmm_MKL_coding_pred K on V.fathmm_MKL_coding_predID=K.fathmm_MKL_coding_predID 
        inner join PredictionStats P on V.VariantID=P.VariantID
        WHERE CHROM=? AND POS =? AND ID=? AND REF =? AND ALT =?
        Group by V.CHROM,V.POS,V.ID,V.REF,V.ALT"""
    
        cursor.execute(sql_statement, values)
        table = cursor.fetchall()    
    #print(table)
        return (table[0])
    
    except:
        sql_statement = 'SELECT * FROM VARIANTS WHERE CHROM=? AND POS =? AND ID=? AND REF =? AND ALT =? '
        
        cursor.execute(sql_statement, values)
        
        data = cursor.fetchall()
        
        #print(data)
        data = list(data[0])
        
        data.append(None)
        
        return tuple(data[1:])
    
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    

In [25]:
db_file = 'lab2.db'
conn = create_connection(db_file)
assert fetch_variant(conn, '22', 25599849, 'rs17670506', 'G', 'A') == ('22', 25599849, 'rs17670506', 'G', 'A', 3124.91, 'PASS', 0.0251597, 0.0425, 'D', 'D', 'T', 'T', 'M', 'D', 'D', 'D', 'D', 'D', 'D', 8.5)
conn.close()

In [26]:
db_file = 'lab2.db'
conn = create_connection(db_file)
assert fetch_variant(conn, 'X', 2836184, 'rs73632976', 'C', 'T') == ('X', 2836184, 'rs73632976', 'C', 'T', 1892.12, 'PASS', None, 0.0427, 'D', 'U', 'D', 'T', 'M', 'P', 'D', 'P', 'P', 'D', 'D', 6.5)
conn.close()

In [27]:
db_file = 'lab2.db'
conn = create_connection(db_file)
assert fetch_variant(conn, '5', 155935708, 'rs45559835', 'G', 'A') == ('5', 155935708, 'rs45559835', 'G', 'A', 1577.12, 'PASS', 0.0189696, 0.0451, 'D', 'D', 'T', 'T', 'L', 'D', 'D', 'P', 'B', 'T', 'D', 5.75)
conn.close()

In [28]:
db_file = 'lab2.db'
conn = create_connection(db_file)
assert fetch_variant(conn, '4', 123416186, '.', 'A', 'G') == ('4', 123416186, '.', 'A', 'G', 23.25, 'PASS', None, None, None, None, None, None, None, None, None, None, None, None, None, None)
conn.close()

# Part 15 (10 Points)
Write a function that returns the variant with the highest predictor score sum. 

Return the variant info the following order:
```
Variants.CHROM,
Variants.POS,
Variants.ID,
Variants.REF,
Variants.ALT,
Variants.QUAL,
Variants.FILTER,
Variants.thousandg2015aug_all,
Variants.ExAC_ALL,
FATHMM_pred.prediction,
LRT_pred.prediction,
MetaLR_pred.prediction,
MetaSVM_pred.prediction,
MutationAssessor_pred.prediction,
MutationTaster_pred.prediction,
PROVEAN_pred.prediction,
Polyphen2_HDIV_pred.prediction,
Polyphen2_HVAR_pred.prediction,
SIFT_pred.prediction,
fathmm_MKL_coding_pred.prediction,
sum(PredictionStats.PredictorValue)
        
```

Again, return the predictor text values and the last column is the sum of the prediction values. 

In [29]:
def variant_with_highest_sum_of_predictor_value(conn):
    """
    See part 15 description 
    """
    cursor = conn.cursor()

    sql_statement = """select CHROM,POS,ID,REF,ALT,QUAL,FILTER,thousandg2015aug_all,Exac_ALL, b.prediction,
    c.prediction,d.prediction,e.prediction,f.prediction,g.prediction,
    h.prediction,i.prediction,j.prediction,k.prediction,l.prediction, m.pred 
    from variants a inner join (select variantid, sum(predictorvalue) as pred 
    from PredictionStats group by 1 order by 2 desc limit 1 ) 
    m on a.variantid=m.variantid inner join fathmm_pred b on a.fathmm_predid=b.fathmm_predid
    inner join LRT_pred c on a.LRT_predid=c.LRT_predid inner join MetaLR_pred d on a.MetaLR_predid=d.MetaLR_predid 
    inner join MetaSVM_pred e on a.MetaSVM_predid=e.MetaSVM_predid inner join MutationAssessor_pred f 
    on a.MutationAssessor_predid=f.MutationAssessor_predid inner join MutationTaster_pred g 
    on a.MutationTaster_predid=g.MutationTaster_predid inner join PROVEAN_pred h on a.PROVEAN_predid=h.PROVEAN_predid 
    inner join Polyphen2_HDIV_pred i on a.Polyphen2_HDIV_predid=i.Polyphen2_HDIV_predid inner join Polyphen2_HVAR_pred j 
    on a.Polyphen2_HVAR_predid = j.Polyphen2_HVAR_predid inner join SIFT_pred k on a.SIFT_predid=k.SIFT_predid inner join 
    fathmm_MKL_coding_pred l on a.fathmm_MKL_coding_predid = l.fathmm_MKL_coding_predid;"""
    
    cursor.execute(sql_statement)
    
    ans = cursor.fetchall()
    return ans[0]
    # YOUR CODE HERE
    #raise NotImplementedError()



In [30]:
db_file = 'lab2.db'
conn = create_connection(db_file)
assert variant_with_highest_sum_of_predictor_value(conn) == ('7', 87837848, '.', 'C', 'A', 418.25, 'PASS', None, None, 'T', 'D', 'D', 'D', 'H', 'D', 'D', 'D', 'D', 'D', 'D', 10.0)
conn.close()